# **Pricing Analysis - Specific Branch Analysis**

### *Race*

In [ ]:
# New Branch Race Analysis
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display
from ipywidgets import Output, VBox, HBox, GridBox, Layout

# Create a FileChooser widget
fc = FileChooser()

# Create an output widget
output = Output()

# Define a function that will be called when a file is selected
def on_file_selected(filechooser):
    # Clear the output widget
    output.clear_output(wait=True)
    
    file_name = filechooser.selected
    
    # Analysis
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import plotly.express as px
    import matplotlib.pyplot as plt
    import textwrap
    from scipy import stats as st
    from bioinfokit.analys import stat
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from IPython.display import display, HTML
    #from IPython.display import clear_output
    from plotly.offline import iplot, init_notebook_mode
    import warnings
    warnings.filterwarnings('ignore', category=FutureWarning)
    init_notebook_mode(connected=True)
    from tqdm.notebook import tqdm

    # formatting of the results section
    BOLD = '<b>'
    UNDERLINE = '<u>'
    ITALIC = '<i>'
    END = '</b></u></i>'

    # reading in data
    all_my_data = pd.read_csv(file_name)

    # Get the unique loan types from the data
    loan_types = all_my_data['LoanType'].unique()

    # Get the unique branch names from the data and sort them alphabetically
    branch_names = sorted(all_my_data['branchname'].unique())

    # Create a dropdown widget with the sorted branch names
    branch_dropdown = widgets.Dropdown(
        options=branch_names,
        value=branch_names[0],
        description='Branch:',
    )
                                    
    # Create a radio button widget with options for statistically significant results
    stat_sign_widget = widgets.ToggleButtons(
        options=[1.0, 0.05, 0.01],
        description='Statistical Significance:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['1%' ,'95% (Default)', '99%'],
    )
                                
    # Create a radio button widget with options for mean diff ranges
    mean_diff_widget = widgets.ToggleButtons(
        options=[0.10, 0.15, 0.20, 0.25],
        description='Mean Diff (BPS):',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['10 BPS (Default)', '15 BPS', '20 BPS', '25 BPS'],
    )

    # Create a radio button widget with options for minimum number of observations
    min_observations_widget = widgets.ToggleButtons(
        options=[2, 10, 30],
        description='Min Observations (Loan Count):',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['2 Loans (Default)', '10 Loans', '30 Loans'],
    )

    # Create a radio button widget with options for showing statistic results
    show_stats_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description='Show Statistical Results:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the stats (Default)', 'Include the stats'],
    )

    # Create a radio button widget with options for showing plots
    show_boxplot_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description='Show Boxplots:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the plots (Default)', 'Include the plots'],
    )

    # Create a radio button widget with options for print list of loans
    print_list_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description='Show List of Loans:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
    )

    def apply_filters(branchname, stat_sign, min_observations, show_stats,  show_boxplot, print_list):
        display(HTML("<br>"))
        # Loop through each loan type and filter the data accordingly
        for loan_type in tqdm(loan_types):
            display(HTML("<br>"))
            display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
            display(HTML("<br>"))
            
            #filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
            all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
            all_data = all_da_data[all_da_data['HmdaActionTaken'] =='Loan Originated']
            big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

            #narrowing down the fields for more accurate analysis
            race_all_data = big_all_data[["LoanId","AIP","Rate_Spread","Race","branchname"]]

            res = stat()
            #creating race filters for white vs protected race
            race_filters = {
                'Black or African American': race_all_data['Race'].isin(['White','Black or African American']),
                'American Alaska or Indian': race_all_data['Race'].isin(['White','American Alaska or Indian']),
                'Native Hawaiian or Other Pacific Islander': race_all_data['Race'].isin(['White','Native Hawaiian or Other Pacific Islander']),
                'Asian': race_all_data['Race'].isin(['White','Asian'])
            }
            purch_filter = big_all_data['Purpose'].isin(['Purchase'])
            refin_filter = big_all_data['Purpose'].isin(['Refinance'])
            
            # Loop through each race filter and group the filtered data by 'branchname' for 'Conventional' Loan Type
            for race, race_filter in race_filters.items():
                #display(HTML("<br>"))
                display(HTML(ITALIC + BOLD + f'{race} Analysis' + END))
                display(HTML("<br>"))
                
                filtered_data = race_all_data[race_filter]
                grouped_data = filtered_data.groupby('branchname')
                
                # Loop through each group and perform a t-test
                for branchname, group in grouped_data:
                    # Add a condition to only analyze the selected branchname
                    if branchname == branch_dropdown.value:
                            
                        # Check if there are exactly two levels in the 'Race' column and enough observations for each level
                        if len(group['Race'].unique()) == 2 and all(group['Race'].value_counts() > 1):
                            res.ttest(df=group, xfac='Race', res='AIP', evar=False, test_type=2)
                            
                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split('\n')
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])
                            
                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])
                            
                            # Round the p-value to 4 decimal places
                            mean_diff = round(mean_diff, 2)
                            p_value = round(p_value, 4)
                            
                            if p_value <= stat_sign and mean_diff >= .10:
                                # Print the Branchname, Mean and P Value, and the full Results without restrictions on p_value and mean_diff_range 
                                display(HTML(BOLD + branchname + END))
                                display(HTML(f'Group: {race}'))
                                display(HTML(f'Loan Type: {loan_type}'))
                                display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                                display(HTML(f'p-value: {p_value}'))
                                display(HTML("<br>"))
                                
                                if show_stats == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                    display(HTML(f'<pre>{res.summary}</pre>'))
                                    display(HTML("<br>"))
                                                        
                                # Create a box chart of the AIP values grouped by Race using plotly.express
                                if show_boxplot == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                    display(HTML("<br>"))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                    plt.show()
                                    display(HTML("<br>"))

                                # Print the list of loans included in the analysis
                                if print_list == 'Include':
                                    group = group.sort_values('Race', ascending=True)
                                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                    
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                    html_table = group.to_html(index=False)
                                        
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br>"))

    # Use the interactive function to create UI controls
    interactive_plot = interactive(apply_filters, branchname=branch_dropdown, stat_sign=stat_sign_widget, min_observations=min_observations_widget, show_stats=show_stats_widget, show_boxplot=show_boxplot_widget, print_list=print_list_widget)

    # Organize the widgets in two columns using VBox
    column1 = VBox(interactive_plot.children[:4])
    column2 = VBox(interactive_plot.children[4:6])

    # Create a grid using HBox and display it
    display(HBox([column1, column2]))

    # Display the output of the function
    display(interactive_plot.children[-1])
    
# Set the function to be called when a file is selected
fc.register_callback(on_file_selected)

# Display the FileChooser widget
display(fc)
# Change the title (use '' to hide)
fc.title = '<b>Select (.csv) File for Race Branch Analysis</b>'

# Display the output widget
display(output)

In [ ]:
# New Branch Race Analysis
from ipyfilechooser import FileChooser
from IPython.display import clear_output, display
from ipywidgets import Output

# Create a FileChooser widget
fc = FileChooser()

# Create an output widget
output = Output()

# Define a function that will be called when a file is selected
def on_file_selected(filechooser):
    # Clear the output widget
    output.clear_output(wait=True)
    
    file_name = filechooser.selected
    
    # Analysis
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import plotly.express as px
    import matplotlib.pyplot as plt
    import textwrap
    from scipy import stats as st
    from bioinfokit.analys import stat
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from IPython.display import display, HTML
    #from IPython.display import clear_output
    from plotly.offline import iplot, init_notebook_mode
    import warnings
    warnings.filterwarnings('ignore', category=FutureWarning)
    init_notebook_mode(connected=True)
    from tqdm.notebook import tqdm

    # formatting of the results section
    BOLD = '<b>'
    UNDERLINE = '<u>'
    ITALIC = '<i>'
    END = '</b></u></i>'

    # reading in data
    all_my_data = pd.read_csv(file_name)

    # Get the unique loan types from the data
    loan_types = all_my_data['LoanType'].unique()

    # Get the unique branch names from the data and sort them alphabetically
    branch_names = sorted(all_my_data['branchname'].unique())

    # Create a dropdown widget with the sorted branch names
    branch_dropdown = widgets.Dropdown(
        options=branch_names,
        value=branch_names[0],
        description='Branch:',
    )
                                    
    # Create a radio button widget with options for statistically significant results
    stat_sign_widget = widgets.ToggleButtons(
        options=[1.0, 0.05, 0.01],
        description='Statistical Significance:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['1%' ,'95% (Default)', '99%'],
    )
                                
    # Create a radio button widget with options for mean diff ranges
    mean_diff_widget = widgets.ToggleButtons(
        options=[0.10, 0.15, 0.20, 0.25],
        description='Mean Diff (BPS):',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['10 BPS (Default)', '15 BPS', '20 BPS', '25 BPS'],
    )

    # Create a radio button widget with options for minimum number of observations
    min_observations_widget = widgets.ToggleButtons(
        options=[2, 10, 30],
        description='Min Observations (Loan Count):',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['2 Loans (Default)', '10 Loans', '30 Loans'],
    )

    # Create a radio button widget with options for showing statistic results
    show_stats_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description='Show Statistical Results:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the stats (Default)', 'Include the stats'],
    )

    # Create a radio button widget with options for showing plots
    show_boxplot_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description='Show Boxplots:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the plots (Default)', 'Include the plots'],
    )

    # Create a radio button widget with options for print list of loans
    print_list_widget = widgets.ToggleButtons(
        options=["Exclude", "Include"],
        description='Show List of Loans:',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
        tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
    )

    def apply_filters(branchname, stat_sign, mean_diff_range, min_observations, show_stats,  show_boxplot, print_list):
        display(HTML("<br>"))
        # Loop through each loan type and filter the data accordingly
        for loan_type in tqdm(loan_types):
            display(HTML("<br>"))
            display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
            display(HTML("<br>"))
            
            #filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
            all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
            all_data = all_da_data[all_da_data['HmdaActionTaken'] =='Loan Originated']
            big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

            #narrowing down the fields for more accurate analysis
            race_all_data = big_all_data[["LoanId","AIP","Rate_Spread","Race","branchname"]]

            res = stat()
            #creating race filters for white vs protected race
            race_filters = {
                'Black or African American': race_all_data['Race'].isin(['White','Black or African American']),
                'American Alaska or Indian': race_all_data['Race'].isin(['White','American Alaska or Indian']),
                'Native Hawaiian or Other Pacific Islander': race_all_data['Race'].isin(['White','Native Hawaiian or Other Pacific Islander']),
                'Asian': race_all_data['Race'].isin(['White','Asian'])
            }
            purch_filter = big_all_data['Purpose'].isin(['Purchase'])
            refin_filter = big_all_data['Purpose'].isin(['Refinance'])
            
            # Loop through each race filter and group the filtered data by 'branchname' for 'Conventional' Loan Type
            for race, race_filter in race_filters.items():
                #display(HTML("<br>"))
                display(HTML(ITALIC + BOLD + f'{race} Analysis' + END))
                display(HTML("<br>"))
                
                filtered_data = race_all_data[race_filter]
                grouped_data = filtered_data.groupby('branchname')
                
                # Loop through each group and perform a t-test
                for branchname, group in grouped_data:
                    # Add a condition to only analyze the selected branchname
                    if branchname == branch_dropdown.value:
                            
                        # Check if there are exactly two levels in the 'Race' column and enough observations for each level
                        if len(group['Race'].unique()) == 2 and all(group['Race'].value_counts() > 1):
                            res.ttest(df=group, xfac='Race', res='AIP', evar=False, test_type=2)
                            
                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split('\n')
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])
                            
                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])
                            
                            # Round the p-value to 4 decimal places
                            mean_diff = round(mean_diff, 2)
                            p_value = round(p_value, 4)
                            
                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                                # Print the Branchname, Mean and P Value, and the full Results without restrictions on p_value and mean_diff_range 
                                display(HTML(BOLD + branchname + END))
                                display(HTML(f'Group: {race}'))
                                display(HTML(f'Loan Type: {loan_type}'))
                                display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                                display(HTML(f'p-value: {p_value}'))
                                display(HTML("<br>"))
                                
                                if show_stats == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                    display(HTML(f'<pre>{res.summary}</pre>'))
                                    display(HTML("<br>"))
                                                        
                                # Create a box chart of the AIP values grouped by Race using plotly.express
                                if show_boxplot == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                    display(HTML("<br>"))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                    plt.show()
                                    display(HTML("<br>"))

                                # Print the list of loans included in the analysis
                                if print_list == 'Include':
                                    group = group.sort_values('Race', ascending=True)
                                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                    
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                    html_table = group.to_html(index=False)
                                        
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br>"))

    # Use the interact function to apply the selected filters to the code
    interact(apply_filters, branchname=branch_dropdown, stat_sign=stat_sign_widget, mean_diff_range=mean_diff_widget, min_observations=min_observations_widget, show_stats=show_stats_widget, show_boxplot=show_boxplot_widget, print_list=print_list_widget);
    
# Set the function to be called when a file is selected
fc.register_callback(on_file_selected)

# Display the FileChooser widget
display(fc)
# Change the title (use '' to hide)
fc.title = '<b>Select (.csv) File for Race Branch Analysis</b>'

# Display the output widget
display(output)

In [ ]:
# File Selection for analysis
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser()
display(fc)


In [ ]:
# Analysis
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
#from IPython.display import clear_output
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

# Check if a file has been selected
if fc.selected is not None:
    file_name = fc.selected
    # The rest of your code goes here...
else:
    print("No file selected. Please select a file.")
    
# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)
                                   
# Create a radio button widget with options for statistically significant results
stat_sign_widget = widgets.ToggleButtons(
    options=[1.0, 0.05, 0.01],
    description='Statistical Significance:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['1%' ,'95% (Default)', '99%'],
)
                               
# Create a radio button widget with options for mean diff ranges
mean_diff_widget = widgets.ToggleButtons(
    options=[0.10, 0.15, 0.20, 0.25],
    description='Mean Diff (BPS):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['10 BPS (Default)', '15 BPS', '20 BPS', '25 BPS'],
)

# Create a radio button widget with options for minimum number of observations
min_observations_widget = widgets.ToggleButtons(
    options=[2, 10, 30],
    description='Min Observations (Loan Count):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['2 Loans (Default)', '10 Loans', '30 Loans'],
)

# Create a radio button widget with options for showing statistic results
show_stats_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Statistical Results:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the stats (Default)', 'Include the stats'],
)

# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the plots (Default)', 'Include the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, stat_sign, mean_diff_range, min_observations, show_stats,  show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))
        
        #filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken'] =='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        race_all_data = big_all_data[["LoanId","AIP","Rate_Spread","Race","branchname"]]

        res = stat()
        #creating race filters for white vs protected race
        race_filters = {
            'Black or African American': race_all_data['Race'].isin(['White','Black or African American']),
            'American Alaska or Indian': race_all_data['Race'].isin(['White','American Alaska or Indian']),
            'Native Hawaiian or Other Pacific Islander': race_all_data['Race'].isin(['White','Native Hawaiian or Other Pacific Islander']),
            'Asian': race_all_data['Race'].isin(['White','Asian'])
        }
        purch_filter = big_all_data['Purpose'].isin(['Purchase'])
        refin_filter = big_all_data['Purpose'].isin(['Refinance'])
        
        # Loop through each race filter and group the filtered data by 'branchname' for 'Conventional' Loan Type
        for race, race_filter in race_filters.items():
            #display(HTML("<br>"))
            display(HTML(ITALIC + BOLD + f'{race} Analysis' + END))
            display(HTML("<br>"))
            
            filtered_data = race_all_data[race_filter]
            grouped_data = filtered_data.groupby('branchname')
            
            # Loop through each group and perform a t-test
            for branchname, group in grouped_data:
                # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
                        
                    # Check if there are exactly two levels in the 'Race' column and enough observations for each level
                    if len(group['Race'].unique()) == 2 and all(group['Race'].value_counts() > 1):
                        res.ttest(df=group, xfac='Race', res='AIP', evar=False, test_type=2)
                         
                        # Extract the Mean Diff from the summary
                        summary = res.summary
                        lines = summary.split('\n')
                        mean_diff_line = lines[4]
                        mean_diff = float(mean_diff_line.split()[-1])
                         
                        # Extract the p-value from the summary
                        p_value_line = lines[8]
                        p_value = float(p_value_line.split()[-1])
                         
                        if p_value <= stat_sign and mean_diff >= mean_diff_range:
                            # Print the Branchname, Mean and P Value, and the full Results without restrictions on p_value and mean_diff_range 
                            display(HTML(BOLD + branchname + END))
                            display(HTML(f'Group: {race}'))
                            display(HTML(f'Loan Type: {loan_type}'))
                            display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                            display(HTML(f'p-value: {p_value}'))
                            display(HTML("<br>"))
                            
                            if show_stats == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                display(HTML(f'<pre>{res.summary}</pre>'))
                                display(HTML("<br>"))
                                                       
                            # Create a box chart of the AIP values grouped by Race using plotly.express
                            if show_boxplot == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                display(HTML("<br>"))
                                plt.figure(figsize=(10, 6))
                                sns.set_style('darkgrid')
                                sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                plt.show()
                                display(HTML("<br>"))

                            # Print the list of loans included in the analysis
                            if print_list == 'Include':
                                group = group.sort_values('Race', ascending=True)
                                display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                
                                # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                html_table = group.to_html(index=False)
                                    
                                # Add inline CSS to adjust the font size
                                html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                display(HTML(html_table))
                                display(HTML("<br>"))

# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, stat_sign=stat_sign_widget, mean_diff_range=mean_diff_widget, min_observations=min_observations_widget, show_stats=show_stats_widget, show_boxplot=show_boxplot_widget, print_list=print_list_widget);

In [ ]:
# This is the original code w/o filechooser
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
#from IPython.display import clear_output
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
file_name = "Apr-Jun 2023 Pricing Race.csv"
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)
                                   
# Create a radio button widget with options for statistically significant results
stat_sign_widget = widgets.ToggleButtons(
    options=[1.0, 0.05, 0.01],
    description='Statistical Significance:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['1%' ,'95% (Default)', '99%'],
)
                               
# Create a radio button widget with options for mean diff ranges
mean_diff_widget = widgets.ToggleButtons(
    options=[0.10, 0.15, 0.20, 0.25],
    description='Mean Diff (BPS):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['10 BPS (Default)', '15 BPS', '20 BPS', '25 BPS'],
)

# Create a radio button widget with options for minimum number of observations
min_observations_widget = widgets.ToggleButtons(
    options=[2, 10, 30],
    description='Min Observations (Loan Count):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['2 Loans (Default)', '10 Loans', '30 Loans'],
)

# Create a radio button widget with options for showing statistic results
show_stats_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Statistical Results:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the stats (Default)', 'Include the stats'],
)

# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the plots (Default)', 'Include the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, stat_sign, mean_diff_range, min_observations, show_stats,  show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))
        
        #filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken'] =='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        race_all_data = big_all_data[["LoanId","AIP","Rate_Spread","Race","branchname"]]

        res = stat()
        #creating race filters for white vs protected race
        race_filters = {
            'Black or African American': race_all_data['Race'].isin(['White','Black or African American']),
            'American Alaska or Indian': race_all_data['Race'].isin(['White','American Alaska or Indian']),
            'Native Hawaiian or Other Pacific Islander': race_all_data['Race'].isin(['White','Native Hawaiian or Other Pacific Islander']),
            'Asian': race_all_data['Race'].isin(['White','Asian'])
        }
        purch_filter = big_all_data['Purpose'].isin(['Purchase'])
        refin_filter = big_all_data['Purpose'].isin(['Refinance'])
        
        # Loop through each race filter and group the filtered data by 'branchname' for 'Conventional' Loan Type
        for race, race_filter in race_filters.items():
            #display(HTML("<br>"))
            display(HTML(ITALIC + BOLD + f'{race} Analysis' + END))
            display(HTML("<br>"))
            
            filtered_data = race_all_data[race_filter]
            grouped_data = filtered_data.groupby('branchname')
            
            # Loop through each group and perform a t-test
            for branchname, group in grouped_data:
                # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
                        
                    # Check if there are exactly two levels in the 'Race' column and enough observations for each level
                    if len(group['Race'].unique()) == 2 and all(group['Race'].value_counts() > 1):
                        res.ttest(df=group, xfac='Race', res='AIP', evar=False, test_type=2)
                         
                        # Extract the Mean Diff from the summary
                        summary = res.summary
                        lines = summary.split('\n')
                        mean_diff_line = lines[4]
                        mean_diff = float(mean_diff_line.split()[-1])
                         
                        # Extract the p-value from the summary
                        p_value_line = lines[8]
                        p_value = float(p_value_line.split()[-1])
                         
                        if p_value <= stat_sign and mean_diff >= mean_diff_range:
                            # Print the Branchname, Mean and P Value, and the full Results without restrictions on p_value and mean_diff_range 
                            display(HTML(BOLD + branchname + END))
                            display(HTML(f'Group: {race}'))
                            display(HTML(f'Loan Type: {loan_type}'))
                            display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                            display(HTML(f'p-value: {p_value}'))
                            display(HTML("<br>"))
                            
                            if show_stats == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                display(HTML(f'<pre>{res.summary}</pre>'))
                                display(HTML("<br>"))
                                                       
                            # Create a box chart of the AIP values grouped by Race using plotly.express
                            if show_boxplot == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                display(HTML("<br>"))
                                plt.figure(figsize=(10, 6))
                                sns.set_style('darkgrid')
                                sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                plt.show()
                                display(HTML("<br>"))

                            # Print the list of loans included in the analysis
                            if print_list == 'Include':
                                group = group.sort_values('Race', ascending=True)
                                display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                
                                # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                html_table = group.to_html(index=False)
                                    
                                # Add inline CSS to adjust the font size
                                html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                display(HTML(html_table))
                                display(HTML("<br>"))



# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, stat_sign=stat_sign_widget, mean_diff_range=mean_diff_widget, min_observations=min_observations_widget, show_stats=show_stats_widget, show_boxplot=show_boxplot_widget, print_list=print_list_widget);

---

### *Ethnicity, Age, Sex*

In [1]:
# File Selection for analysis
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser()
display(fc)


FileChooser(path='C:\Users\colby.kellersberger\Documents\Fair Lending Analysis\Pricing', filename='', title=''…

In [ ]:
# Analysis
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
#from IPython.display import clear_output
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

# Check if a file has been selected
if fc.selected is not None:
    file_name = fc.selected
    # The rest of your code goes here...
else:
    print("No file selected. Please select a file.")
    
np.seterr(divide='ignore', invalid='ignore')

init_notebook_mode(connected=True)

# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)

# Create a radio button widget with options for statistically significant results
stat_sign_widget = widgets.ToggleButtons(
    options=[1.0, 0.05, 0.01],
    description='Statistical Significance:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['1%' ,'95% (Default)', '99%'],
)
                               
# Create a radio button widget with options for mean diff ranges
mean_diff_widget = widgets.ToggleButtons(
    options=[0.10, 0.15, 0.20, 0.25],
    description='Mean Diff (BPS):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['10 BPS (Default)', '15 BPS', '20 BPS', '25 BPS'],
)

# Create a radio button widget with options for minimum number of observations
min_observations_widget = widgets.ToggleButtons(
    options=[2, 10, 30],
    description='Min Observations (Loan Count):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['2 Loans (Default)', '10 Loans', '30 Loans'],
)

# Create a radio button widget with options for showing statistic results
show_stats_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Statistical Results:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the stats (Default)', 'Include the stats'],
)

# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the plots (Default)', 'Include the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, stat_sign, mean_diff_range, min_observations, show_stats, show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))

        #filtering to only originated loans and non-HECM programs (Loan Programs: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken']=='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        gender_all_data = big_all_data[["LoanId","AIP","Sex","branchname","Rate_Spread"]]
        age_all_data = big_all_data[["LoanId","AIP","Age","branchname","Rate_Spread"]]
        ethn_all_data = big_all_data[["LoanId","AIP","Class","Ethnicity","branchname","Rate_Spread"]]

        res = stat()
        #creating filters
        #Gender Filter
        gender_filter = gender_all_data['Sex'].isin(['Male','Female'])

        # Filter out rows with 'Exclude' in the 'Class' column
        ethn_all_data = ethn_all_data[ethn_all_data['Class'] != 'EXCLUDED']

        # Create a boolean mask to exclude rows with 'Excluded' in the 'Sex' column
        gender_mask = gender_all_data['Sex'] != 'Excluded'

        # Apply the mask to the data frame
        gender_all_data = gender_all_data[gender_mask]

        #applying filters
        gender = gender_all_data[gender_filter]

        # Group the SEX data by 'branchname'
        grouped_gender_data = gender.groupby('branchname')
        display(HTML(ITALIC + BOLD + f'Sex Analysis' + END))
        display(HTML("<br>"))

        # Loop through each group and perform a t-test
        for branchname, group in grouped_gender_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:

                # Check if there are exactly two levels in the 'Class' column
                    if len(group['Sex'].unique()) == 2:
                        if all(group['Sex'].value_counts() > 1):  # Ensure that each group has more than one observation before performing t-test.
                            res.ttest(df=group, xfac='Sex', res='AIP', evar=False, test_type=2)

                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split('\n')
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])

                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])

                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                            # Print the Branchname, Mean and P Value, and the full Results
                                display(HTML(BOLD + branchname + END))
                                display(HTML(f'Group: Female'))
                                display(HTML(f'Loan Type: {loan_type}'))
                                display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                                display(HTML(f'p-value: {p_value}'))
                                display(HTML("<br>"))
                                
                                if show_stats == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                    display(HTML(f'<pre>{res.summary}</pre>'))
                                    display(HTML("<br>"))

                                # Create a box chart of the AIP values grouped by Sex using plotly.express
                                if show_boxplot == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Sex', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Sex', y='AIP', hue='Sex', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                    plt.show()
                                    display(HTML("<br>")) 

                                # Print the list of loans included in the analysis
                                if print_list == 'Include':
                                    group = group.sort_values('Sex', ascending=False)
                                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                    
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index plt
                                    html_table = group.to_html(index=False)
                                    
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br>"))
                                    
        # Group the AGE data by 'branchname'
        grouped_age_data = age_all_data.groupby('branchname')
        display(HTML("<br>"))
        display(HTML(ITALIC + BOLD + f'Age Analysis' + END))
        display(HTML("<br>"))

        # Loop through each group and perform a t-test
        for branchname, group in grouped_age_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:

                    # Check if there are exactly two levels in the 'Age' column
                    if len(group['Age'].unique()) != 2:
                        continue

                    # Check if there are enough observations for each level
                    if all(group['Age'].value_counts() > 1):  # Ensure that each group has more than one observation before performing t-test.
                        res.ttest(df=group, xfac='Age', res='AIP', evar=False, test_type=2)

                        # Extract the Mean Diff from the summary
                        summary = res.summary
                        lines = summary.split('\n')
                        mean_diff_line = lines[4]
                        mean_diff = float(mean_diff_line.split()[-1])

                        # Extract the p-value from the summary
                        p_value_line = lines[8]
                        p_value = float(p_value_line.split()[-1])

                        if p_value <= stat_sign and mean_diff >= mean_diff_range:
                        # Print the Branchname, Mean and P Value, and the full Results
                            display(HTML(BOLD + branchname + END))
                            display(HTML(f'Group: Seniors'))
                            display(HTML(f'Loan Type: {loan_type}'))
                            display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                            display(HTML(f'p-value: {p_value}'))
                            display(HTML("<br>"))
                            
                            if show_stats == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                display(HTML(f'<pre>{res.summary}</pre>'))
                                display(HTML("<br>"))

                            # Create a box chart of the AIP values grouped by Sex using plotly.express
                            if show_boxplot == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                display(HTML("<br>"))
                                plt.figure(figsize=(10, 6))
                                sns.set_style('darkgrid')
                                sns.boxplot(x='Age', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                sns.stripplot(x='Age', y='AIP', hue='Age', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                plt.show()
                                display(HTML("<br>"))     

                            # Print the list of loans included in the analysis
                            if print_list == 'Include':
                                group = group.sort_values('Age', ascending=False)
                                display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                
                                # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                html_table = group.to_html(index=False)
                                
                                # Add inline CSS to adjust the font size
                                html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                display(HTML(html_table))
                                display(HTML("<br>"))
                               
        # Group the ETHNICITY data by 'branchname'
        grouped_ethn_data = ethn_all_data.groupby('branchname')
        display(HTML("<br>"))
        display(HTML(BOLD + ITALIC + f'Ethnicity Analysis' + END))
        display(HTML("<br>"))

        # Loop through each group and perform a t-test
        for branchname, group in grouped_ethn_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
            
                    # Check if there are exactly two levels in the 'Class' column
                    if len(group['Class'].unique()) == 2:
                        
                        # Check if there are enough observations for each level
                        if all(group['Class'].value_counts() > 1):  # Ensure that each group has more than one observation before performing t-test.
                            res.ttest(df=group, xfac='Class', res='AIP', evar=False, test_type=2)
                            
                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split('\n')
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])
                        
                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])

                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                                # Print the Branchname, Mean and P Value, and the full Results
                                display(HTML(BOLD + branchname + END))
                                display(HTML(f'Group: Hispanic or Latino'))
                                display(HTML(f'Loan Type: {loan_type}'))
                                display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                                display(HTML(f'p-value: {p_value}'))
                                display(HTML("<br>"))
                                
                                if show_stats == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                    display(HTML(f'<pre>{res.summary}</pre>'))
                                    display(HTML("<br>"))
                                                                
                                # Create a box chart of the AIP values grouped by Sex using plotly.express
                                if show_boxplot == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                    display(HTML("<br>"))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Class', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Class', y='AIP', hue='Class', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                    plt.show()
                                    display(HTML("<br>"))     

                                # Print the list of loans included in the analysis
                                if print_list == 'Include':
                                    group = group.sort_values('Class', ascending=False)
                                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                    
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                    html_table = group.to_html(index=False)
                                    
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br>"))
                                    display(HTML(f'<pre>{group.to_string()}</pre>'))
                                    display(HTML("<br>"))
                    
# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, stat_sign=stat_sign_widget, mean_diff_range=mean_diff_widget, min_observations=min_observations_widget, show_stats=show_stats_widget, show_boxplot=show_boxplot_widget, print_list=print_list_widget);


In [ ]:
# This is the original code w/o filechooser
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual, Output, IntProgress
import ipywidgets as widgets
from IPython.display import display, HTML
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

np.seterr(divide='ignore', invalid='ignore')

init_notebook_mode(connected=True)

# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
file_name = "Apr-Jun 2023 Pricing Ethnicity, Age, Sex.csv"
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)

# Create a radio button widget with options for statistically significant results
stat_sign_widget = widgets.ToggleButtons(
    options=[1.0, 0.05, 0.01],
    description='Statistical Significance:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['1%' ,'95% (Default)', '99%'],
)
                               
# Create a radio button widget with options for mean diff ranges
mean_diff_widget = widgets.ToggleButtons(
    options=[0.10, 0.15, 0.20, 0.25],
    description='Mean Diff (BPS):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['10 BPS (Default)', '15 BPS', '20 BPS', '25 BPS'],
)

# Create a radio button widget with options for minimum number of observations
min_observations_widget = widgets.ToggleButtons(
    options=[2, 10, 30],
    description='Min Observations (Loan Count):',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['2 Loans (Default)', '10 Loans', '30 Loans'],
)

# Create a radio button widget with options for showing statistic results
show_stats_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Statistical Results:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the stats (Default)', 'Include the stats'],
)

# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the plots (Default)', 'Include the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, stat_sign, mean_diff_range, min_observations, show_stats, show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))

        #filtering to only originated loans and non-HECM programs (Loan Programs: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken']=='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        gender_all_data = big_all_data[["LoanId","AIP","Sex","branchname","Rate_Spread"]]
        age_all_data = big_all_data[["LoanId","AIP","Age","branchname","Rate_Spread"]]
        ethn_all_data = big_all_data[["LoanId","AIP","Class","Ethnicity","branchname","Rate_Spread"]]

        res = stat()
        #creating filters
        #Gender Filter
        gender_filter = gender_all_data['Sex'].isin(['Male','Female'])

        # Filter out rows with 'Exclude' in the 'Class' column
        ethn_all_data = ethn_all_data[ethn_all_data['Class'] != 'EXCLUDED']

        # Create a boolean mask to exclude rows with 'Excluded' in the 'Sex' column
        gender_mask = gender_all_data['Sex'] != 'Excluded'

        # Apply the mask to the data frame
        gender_all_data = gender_all_data[gender_mask]

        #applying filters
        gender = gender_all_data[gender_filter]

        # Group the SEX data by 'branchname'
        grouped_gender_data = gender.groupby('branchname')
        display(HTML(ITALIC + BOLD + f'Sex Analysis' + END))
        display(HTML("<br>"))

        # Loop through each group and perform a t-test
        for branchname, group in grouped_gender_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:

                # Check if there are exactly two levels in the 'Class' column
                    if len(group['Sex'].unique()) == 2:
                        if all(group['Sex'].value_counts() > 1):  # Ensure that each group has more than one observation before performing t-test.
                            res.ttest(df=group, xfac='Sex', res='AIP', evar=False, test_type=2)

                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split('\n')
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])

                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])

                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                            # Print the Branchname, Mean and P Value, and the full Results
                                display(HTML(BOLD + branchname + END))
                                display(HTML(f'Group: Female'))
                                display(HTML(f'Loan Type: {loan_type}'))
                                display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                                display(HTML(f'p-value: {p_value}'))
                                display(HTML("<br>"))
                                
                                if show_stats == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                    display(HTML(f'<pre>{res.summary}</pre>'))
                                    display(HTML("<br>"))

                                # Create a box chart of the AIP values grouped by Sex using plotly.express
                                if show_boxplot == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Sex', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Sex', y='AIP', hue='Sex', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                    plt.show()
                                    display(HTML("<br>")) 

                                # Print the list of loans included in the analysis
                                if print_list == 'Include':
                                    group = group.sort_values('Sex', ascending=False)
                                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                    
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index plt
                                    html_table = group.to_html(index=False)
                                    
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br>"))
                                    
        # Group the AGE data by 'branchname'
        grouped_age_data = age_all_data.groupby('branchname')
        display(HTML("<br>"))
        display(HTML(ITALIC + BOLD + f'Age Analysis' + END))
        display(HTML("<br>"))

        # Loop through each group and perform a t-test
        for branchname, group in grouped_age_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:

                    # Check if there are exactly two levels in the 'Age' column
                    if len(group['Age'].unique()) != 2:
                        continue

                    # Check if there are enough observations for each level
                    if all(group['Age'].value_counts() > 1):  # Ensure that each group has more than one observation before performing t-test.
                        res.ttest(df=group, xfac='Age', res='AIP', evar=False, test_type=2)

                        # Extract the Mean Diff from the summary
                        summary = res.summary
                        lines = summary.split('\n')
                        mean_diff_line = lines[4]
                        mean_diff = float(mean_diff_line.split()[-1])

                        # Extract the p-value from the summary
                        p_value_line = lines[8]
                        p_value = float(p_value_line.split()[-1])

                        if p_value <= stat_sign and mean_diff >= mean_diff_range:
                        # Print the Branchname, Mean and P Value, and the full Results
                            display(HTML(BOLD + branchname + END))
                            display(HTML(f'Group: Seniors'))
                            display(HTML(f'Loan Type: {loan_type}'))
                            display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                            display(HTML(f'p-value: {p_value}'))
                            display(HTML("<br>"))
                            
                            if show_stats == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                display(HTML(f'<pre>{res.summary}</pre>'))
                                display(HTML("<br>"))

                            # Create a box chart of the AIP values grouped by Sex using plotly.express
                            if show_boxplot == 'Include':
                                display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                display(HTML("<br>"))
                                plt.figure(figsize=(10, 6))
                                sns.set_style('darkgrid')
                                sns.boxplot(x='Age', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                sns.stripplot(x='Age', y='AIP', hue='Age', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                plt.show()
                                display(HTML("<br>"))     

                            # Print the list of loans included in the analysis
                            if print_list == 'Include':
                                group = group.sort_values('Age', ascending=False)
                                display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                
                                # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                html_table = group.to_html(index=False)
                                
                                # Add inline CSS to adjust the font size
                                html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                display(HTML(html_table))
                                display(HTML("<br>"))
                               
        # Group the ETHNICITY data by 'branchname'
        grouped_ethn_data = ethn_all_data.groupby('branchname')
        display(HTML("<br>"))
        display(HTML(BOLD + ITALIC + f'Ethnicity Analysis' + END))
        display(HTML("<br>"))

        # Loop through each group and perform a t-test
        for branchname, group in grouped_ethn_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
            
                    # Check if there are exactly two levels in the 'Class' column
                    if len(group['Class'].unique()) == 2:
                        
                        # Check if there are enough observations for each level
                        if all(group['Class'].value_counts() > 1):  # Ensure that each group has more than one observation before performing t-test.
                            res.ttest(df=group, xfac='Class', res='AIP', evar=False, test_type=2)
                            
                            # Extract the Mean Diff from the summary
                            summary = res.summary
                            lines = summary.split('\n')
                            mean_diff_line = lines[4]
                            mean_diff = float(mean_diff_line.split()[-1])
                        
                            # Extract the p-value from the summary
                            p_value_line = lines[8]
                            p_value = float(p_value_line.split()[-1])

                            if p_value <= stat_sign and mean_diff >= mean_diff_range:
                                # Print the Branchname, Mean and P Value, and the full Results
                                display(HTML(BOLD + branchname + END))
                                display(HTML(f'Group: Hispanic or Latino'))
                                display(HTML(f'Loan Type: {loan_type}'))
                                display(HTML(f'BPS Diff: {BOLD}{mean_diff}{END}'))
                                display(HTML(f'p-value: {p_value}'))
                                display(HTML("<br>"))
                                
                                if show_stats == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Statistical Results' + END))
                                    display(HTML(f'<pre>{res.summary}</pre>'))
                                    display(HTML("<br>"))
                                                                
                                # Create a box chart of the AIP values grouped by Sex using plotly.express
                                if show_boxplot == 'Include':
                                    display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                                    display(HTML("<br>"))
                                    plt.figure(figsize=(10, 6))
                                    sns.set_style('darkgrid')
                                    sns.boxplot(x='Class', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                                    sns.stripplot(x='Class', y='AIP', hue='Class', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                                    plt.show()
                                    display(HTML("<br>"))     

                                # Print the list of loans included in the analysis
                                if print_list == 'Include':
                                    group = group.sort_values('Class', ascending=False)
                                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                                    
                                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                                    html_table = group.to_html(index=False)
                                    
                                    # Add inline CSS to adjust the font size
                                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                                    display(HTML(html_table))
                                    display(HTML("<br>"))
                                    display(HTML(f'<pre>{group.to_string()}</pre>'))
                                    display(HTML("<br>"))
                    
# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, stat_sign=stat_sign_widget, mean_diff_range=mean_diff_widget, min_observations=min_observations_widget, show_stats=show_stats_widget, show_boxplot=show_boxplot_widget, print_list=print_list_widget);


---

### *Race Combined Plots*

In [ ]:
# File Selection for analysis
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser()
display(fc)


In [ ]:
# Analysis
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
#from IPython.display import clear_output
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

# Check if a file has been selected
if fc.selected is not None:
    file_name = fc.selected
    # The rest of your code goes here...
else:
    print("No file selected. Please select a file.")

# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)
                                   
# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Include", "Exclude"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Include the plots (Default)', 'Exclude the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))
        
        #filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken'] =='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        race_all_data = big_all_data[["LoanId","AIP","Rate_Spread","Race","branchname"]]
          
        #filtered_data = race_all_data[race_filter]
        grouped_data = race_all_data.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_data:
            # Add a condition to only analyze the selected branchname
            if branchname == branch_dropdown.value:
                
                loan_count = len(group)
                display(HTML(f'{BOLD} Total Loan Count: {loan_count}') )
                    
                # Create a box chart of the AIP values grouped by Race using plotly.express
                if show_boxplot == 'Include':
                    #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                    display(HTML("<br>"))
                    plt.figure(figsize=(10, 6))
                    sns.set_style('darkgrid')
                    sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                    sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                    plt.show()
                    display(HTML("<br>"))

                # Print the list of loans included in the analysis
                if print_list == 'Include':
                    group = group.sort_values('Race', ascending=True)
                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                    
                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                    html_table = group.to_html(index=False)
                        
                    # Add inline CSS to adjust the font size
                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                    display(HTML(html_table))
                    display(HTML("<br>"))

# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, show_boxplot=show_boxplot_widget, print_list=print_list_widget);


In [ ]:
# this is the original code w/o filechooser
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
#from IPython.display import clear_output
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
file_name = "Apr-Jun 2023 Pricing Race.csv"
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)
                                   
# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Include", "Exclude"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Include the plots (Default)', 'Exclude the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))
        
        #filtering to only originated loans and non-HECM programs (Loan Type: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken'] =='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        race_all_data = big_all_data[["LoanId","AIP","Rate_Spread","Race","branchname"]]
          
        #filtered_data = race_all_data[race_filter]
        grouped_data = race_all_data.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_data:
            # Add a condition to only analyze the selected branchname
            if branchname == branch_dropdown.value:
                
                loan_count = len(group)
                display(HTML(f'{BOLD} Total Loan Count: {loan_count}') )
                    
                # Create a box chart of the AIP values grouped by Race using plotly.express
                if show_boxplot == 'Include':
                    #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                    display(HTML("<br>"))
                    plt.figure(figsize=(10, 6))
                    sns.set_style('darkgrid')
                    sns.boxplot(x='Race', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                    sns.stripplot(x='Race', y='AIP', hue='Race', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                    plt.show()
                    display(HTML("<br>"))

                # Print the list of loans included in the analysis
                if print_list == 'Include':
                    group = group.sort_values('Race', ascending=True)
                    display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                    
                    # Use to_html() to format the DataFrame as an HTML table and exclude the index
                    html_table = group.to_html(index=False)
                        
                    # Add inline CSS to adjust the font size
                    html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                    display(HTML(html_table))
                    display(HTML("<br>"))

# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, show_boxplot=show_boxplot_widget, print_list=print_list_widget);

---

### *Ethnicity, Age, Sex Combined Boxplots*

In [ ]:
# File Selection for analysis
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser()
display(fc)

In [ ]:
# Analysis
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import textwrap
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML
#from IPython.display import clear_output
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from tqdm.notebook import tqdm

# Check if a file has been selected
if fc.selected is not None:
    file_name = fc.selected
    # The rest of your code goes here...
else:
    print("No file selected. Please select a file.")
    
# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)

# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Include", "Exclude"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the plots (Default)', 'Include the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))

        #filtering to only originated loans and non-HECM programs (Loan Programs: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken']=='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        gender_all_data = big_all_data[["LoanId","AIP","Sex","branchname","Rate_Spread"]]
        age_all_data = big_all_data[["LoanId","AIP","Age","branchname","Rate_Spread"]]
        ethn_all_data = big_all_data[["LoanId","AIP","Class","Ethnicity","branchname","Rate_Spread"]]

        res = stat()
        #creating filters
        #Gender Filter
        gender_filter = gender_all_data['Sex'].isin(['Male','Female'])

        # Filter out rows with 'Exclude' in the 'Class' column
        ethn_all_data = ethn_all_data[ethn_all_data['Class'] != 'EXCLUDED']

        # Create a boolean mask to exclude rows with 'Excluded' in the 'Sex' column
        gender_mask = gender_all_data['Sex'] != 'Excluded'

        # Apply the mask to the data frame
        gender_all_data = gender_all_data[gender_mask]

        #applying filters
        gender = gender_all_data[gender_filter]

        # Group the SEX data by 'branchname'
        grouped_gender_data = gender.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_gender_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
                    
                    loan_count = len(group)
                    display(HTML(f'{BOLD} Total Loan Count for Sex: {loan_count}') )

                    # Create a box chart of the AIP values grouped by Sex using plotly.express
                    if show_boxplot == 'Include':
                        #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                        plt.figure(figsize=(10, 6))
                        sns.set_style('darkgrid')
                        sns.boxplot(x='Sex', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                        sns.stripplot(x='Sex', y='AIP', hue='Sex', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                        plt.show()
                        display(HTML("<br>")) 

                    # Print the list of loans included in the analysis
                    if print_list == 'Include':
                        group = group.sort_values('Sex', ascending=False)
                        display(HTML("<br>"))
                        display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                        
                        # Use to_html() to format the DataFrame as an HTML table and exclude the index
                        html_table = group.to_html(index=False)
                        
                        # Add inline CSS to adjust the font size
                        html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                        display(HTML(html_table))
                        display(HTML("<br>"))
                                    
        # Group the AGE data by 'branchname'
        grouped_age_data = age_all_data.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_age_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
                    
                    loan_count = len(group)
                    display(HTML(f'{BOLD} Total Loan Count for Age: {loan_count}') )

                    # Create a box chart of the AIP values grouped by Sex using plotly.express
                    if show_boxplot == 'Include':
                        #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                        display(HTML("<br>"))
                        plt.figure(figsize=(10, 6))
                        sns.set_style('darkgrid')
                        sns.boxplot(x='Age', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                        sns.stripplot(x='Age', y='AIP', hue='Age', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                        plt.show()
                        display(HTML("<br>"))     

                    # Print the list of loans included in the analysis
                    if print_list == 'Include':
                        group = group.sort_values('Age', ascending=False)
                        display(HTML("<br>"))
                        display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                        
                        # Use to_html() to format the DataFrame as an HTML table and exclude the index
                        html_table = group.to_html(index=False)
                        
                        # Add inline CSS to adjust the font size
                        html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                        display(HTML(html_table))
                        display(HTML("<br>"))
                               
        # Group the ETHNICITY data by 'branchname'
        grouped_ethn_data = ethn_all_data.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_ethn_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
            
                    loan_count = len(group)
                    display(HTML(f'{BOLD} Total Loan Count for Ethnicity: {loan_count}') )
                    
                    # Create a box chart of the AIP values grouped by Sex using plotly.express
                    if show_boxplot == 'Include':
                        #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                        display(HTML("<br>"))
                        plt.figure(figsize=(10, 6))
                        sns.set_style('darkgrid')
                        sns.boxplot(x='Class', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                        sns.stripplot(x='Class', y='AIP', hue='Class', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                        plt.show()
                        display(HTML("<br>"))     

                    # Print the list of loans included in the analysis
                    if print_list == 'Include':
                        group = group.sort_values('Class', ascending=False)
                        display(HTML("<br>"))
                        display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                        
                        # Use to_html() to format the DataFrame as an HTML table and exclude the index
                        html_table = group.to_html(index=False)
                        
                        # Add inline CSS to adjust the font size
                        html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                        display(HTML(html_table))
                        display(HTML("<br>"))
                        
                        
# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, show_boxplot=show_boxplot_widget, print_list=print_list_widget);


In [ ]:
# This is the original code w/o filechooser
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st
from bioinfokit.analys import stat
from ipywidgets import interact, interactive, fixed, interact_manual, Output, IntProgress
import ipywidgets as widgets
from IPython.display import display, HTML
from plotly.offline import iplot, init_notebook_mode
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

init_notebook_mode(connected=True)

# formatting of the results section
BOLD = '<b>'
UNDERLINE = '<u>'
ITALIC = '<i>'
END = '</b></u></i>'

# reading in data
file_name = "Apr-Jun 2023 Pricing Ethnicity, Age, Sex.csv"
all_my_data = pd.read_csv(file_name)

# remove .csv from file_name
file_name_no_ext = file_name.split('.csv')[0]

# display
display(HTML(BOLD + UNDERLINE + file_name_no_ext + END))

# Get the unique loan types from the data
loan_types = all_my_data['LoanType'].unique()

# Get the unique branch names from the data and sort them alphabetically
branch_names = sorted(all_my_data['branchname'].unique())

# Create a dropdown widget with the sorted branch names
branch_dropdown = widgets.Dropdown(
    options=branch_names,
    value=branch_names[0],
    description='Branch:',
)

# Create a radio button widget with options for showing plots
show_boxplot_widget = widgets.ToggleButtons(
    options=["Include", "Exclude"],
    description='Show Boxplots:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the plots (Default)', 'Include the plots'],
)

# Create a radio button widget with options for print list of loans
print_list_widget = widgets.ToggleButtons(
    options=["Exclude", "Include"],
    description='Show List of Loans:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Exclude the list of loans (Default)', 'Include the list of loans'],
)

def apply_filters(branchname, show_boxplot, print_list):
    # Loop through each loan type and filter the data accordingly
    for loan_type in tqdm(loan_types):
        display(HTML("<br>"))
        display(HTML(UNDERLINE + BOLD + f'Loan Type: {loan_type}' + END))
        display(HTML("<br>"))

        #filtering to only originated loans and non-HECM programs (Loan Programs: 'Conventional')
        all_da_data = all_my_data[all_my_data['LoanType'] == loan_type]
        all_data = all_da_data[all_da_data['HmdaActionTaken']=='Loan Originated']
        big_all_data = all_data[all_data['Program'].str.contains("HECM")==False]

        #narrowing down the fields for more accurate analysis
        gender_all_data = big_all_data[["LoanId","AIP","Sex","branchname","Rate_Spread"]]
        age_all_data = big_all_data[["LoanId","AIP","Age","branchname","Rate_Spread"]]
        ethn_all_data = big_all_data[["LoanId","AIP","Class","Ethnicity","branchname","Rate_Spread"]]

        res = stat()
        #creating filters
        #Gender Filter
        gender_filter = gender_all_data['Sex'].isin(['Male','Female'])

        # Filter out rows with 'Exclude' in the 'Class' column
        ethn_all_data = ethn_all_data[ethn_all_data['Class'] != 'EXCLUDED']

        # Create a boolean mask to exclude rows with 'Excluded' in the 'Sex' column
        gender_mask = gender_all_data['Sex'] != 'Excluded'

        # Apply the mask to the data frame
        gender_all_data = gender_all_data[gender_mask]

        #applying filters
        gender = gender_all_data[gender_filter]

        # Group the SEX data by 'branchname'
        grouped_gender_data = gender.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_gender_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
                    
                    loan_count = len(group)
                    display(HTML(f'{BOLD} Total Loan Count for Sex: {loan_count}') )

                    # Create a box chart of the AIP values grouped by Sex using plotly.express
                    if show_boxplot == 'Include':
                        #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                        plt.figure(figsize=(10, 6))
                        sns.set_style('darkgrid')
                        sns.boxplot(x='Sex', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                        sns.stripplot(x='Sex', y='AIP', hue='Sex', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                        plt.show()
                        display(HTML("<br>")) 

                    # Print the list of loans included in the analysis
                    if print_list == 'Include':
                        group = group.sort_values('Sex', ascending=False)
                        display(HTML("<br>"))
                        display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                        
                        # Use to_html() to format the DataFrame as an HTML table and exclude the index
                        html_table = group.to_html(index=False)
                        
                        # Add inline CSS to adjust the font size
                        html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                        display(HTML(html_table))
                        display(HTML("<br>"))
                                    
        # Group the AGE data by 'branchname'
        grouped_age_data = age_all_data.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_age_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
                    
                    loan_count = len(group)
                    display(HTML(f'{BOLD} Total Loan Count for Age: {loan_count}') )

                    # Create a box chart of the AIP values grouped by Sex using plotly.express
                    if show_boxplot == 'Include':
                        #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                        display(HTML("<br>"))
                        plt.figure(figsize=(10, 6))
                        sns.set_style('darkgrid')
                        sns.boxplot(x='Age', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                        sns.stripplot(x='Age', y='AIP', hue='Age', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                        plt.show()
                        display(HTML("<br>"))     

                    # Print the list of loans included in the analysis
                    if print_list == 'Include':
                        group = group.sort_values('Age', ascending=False)
                        display(HTML("<br>"))
                        display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                        
                        # Use to_html() to format the DataFrame as an HTML table and exclude the index
                        html_table = group.to_html(index=False)
                        
                        # Add inline CSS to adjust the font size
                        html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                        display(HTML(html_table))
                        display(HTML("<br>"))
                               
        # Group the ETHNICITY data by 'branchname'
        grouped_ethn_data = ethn_all_data.groupby('branchname')
        
        # Loop through each group and perform a t-test
        for branchname, group in grouped_ethn_data:
            # Add a condition to only analyze the selected branchname
                if branchname == branch_dropdown.value:
            
                    loan_count = len(group)
                    display(HTML(f'{BOLD} Total Loan Count for Ethnicity: {loan_count}') )
                    
                    # Create a box chart of the AIP values grouped by Sex using plotly.express
                    if show_boxplot == 'Include':
                        #display(HTML(BOLD + UNDERLINE + f'Loan Pricing Plot' + END))
                        display(HTML("<br>"))
                        plt.figure(figsize=(10, 6))
                        sns.set_style('darkgrid')
                        sns.boxplot(x='Class', y='AIP', data=group, color='skyblue', showmeans=True, fliersize=0, meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor":"black", "markersize":"10"})
                        sns.stripplot(x='Class', y='AIP', hue='Class', data=group, jitter=0.4, dodge=False, linewidth=1, palette="Dark2", legend=True, size=8)
                        plt.show()
                        display(HTML("<br>"))     

                    # Print the list of loans included in the analysis
                    if print_list == 'Include':
                        group = group.sort_values('Class', ascending=False)
                        display(HTML("<br>"))
                        display(HTML(BOLD + UNDERLINE + f'List of Loans' + END))
                        
                        # Use to_html() to format the DataFrame as an HTML table and exclude the index
                        html_table = group.to_html(index=False)
                        
                        # Add inline CSS to adjust the font size
                        html_table = '<style> table {font-size: 0.9em;} </style>' + html_table
                        display(HTML(html_table))
                        display(HTML("<br>"))
                        
                        
# Use the interact function to apply the selected filters to the code
interact(apply_filters, branchname=branch_dropdown, show_boxplot=show_boxplot_widget, print_list=print_list_widget);


---